In [6]:
%pip install -U git+https://github.com/ElvisCasco/hw5.git@main

  Cloning https://github.com/ElvisCasco/hw5.git (to revision main) to /private/var/folders/f0/99976rv943qfw3wg20pdpchh0000gn/T/pip-req-build-889l_jxl
  Running command git clone --filter=blob:none --quiet https://github.com/ElvisCasco/hw5.git /private/var/folders/f0/99976rv943qfw3wg20pdpchh0000gn/T/pip-req-build-889l_jxl
  Resolved https://github.com/ElvisCasco/hw5.git to commit 55efceb71cef228b07cddeb7f87c07b63df09981
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 16.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 17.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 15.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.0 MB/s eta 0:00:00a 0:00:01
  Created wheel 

In [1]:
from process_data import (
    CSVDataLoader, DataPreprocessor,
    BMICalculator, EthnicityEncoder, GenderBinaryEncoder,
    DiabetesModel,
)
from sklearn.metrics import roc_auc_score

In [3]:
file_name = "sample_diabetes_mellitus_data.csv"

loader = CSVDataLoader(file_name)
train_df, test_df = loader.split_data()

pre = DataPreprocessor(train_df, test_df)
train_clean, test_clean = (
    pre.remove_nans(['age','gender','ethnicity'])
      .fill_nans_with_mean(['height','weight'])
      .get_data()
)

for t in [BMICalculator(), EthnicityEncoder(), GenderBinaryEncoder()]:
    train_clean = t.transform(train_clean)
    test_clean = t.transform(test_clean)


Data loaded. Shape: (10000, 53)

Data split: 
Train ((8000, 53)), 
Test ((2000, 53))

Data after removing rows with NaNs in ['age', 'gender', 'ethnicity']: 
Train ((7494, 53)), 
Test ((1874, 53))

Filled NaNs in ['height', 'weight'] with mean.



In [4]:
feature_cols = ['age', 'bmi', 'gender_M', 'gender_F'] + [c for c in train_clean.columns if c.startswith('ethnicity_')]
target_col = 'diabetes_mellitus'

model = DiabetesModel(feature_columns=feature_cols, target_column=target_col, hyperparameters={'n_estimators':200,'max_depth':10,'random_state':42})
model.train(train_clean)

probs = model.predict(test_clean)
pos = probs[:,1] if probs.ndim == 2 else probs
test_clean = test_clean.copy()
test_clean['predictions_prob'] = pos
test_clean['predictions'] = (pos >= 0.5).astype(int)

roc_auc_score(test_clean[target_col], test_clean['predictions_prob'])

np.float64(0.6611063857899039)

In [10]:
probs = model.predict(test_clean)
pos = probs[:, 1] if probs.ndim == 2 else probs
test_clean['predictions_prob'] = pos
test_clean['predictions'] = (pos >= 0.5).astype(int)


print(test_clean[['predictions_prob', 'predictions']].head())


      predictions_prob  predictions
6252          0.371048            0
1731          0.203191            0
4742          0.112263            0
4521          0.169594            0
6340          0.319036            0


In [9]:
test_clean.columns

Index(['Unnamed: 0', 'encounter_id', 'hospital_id', 'age', 'bmi',
       'elective_surgery', 'ethnicity', 'gender', 'height',
       'hospital_admit_source', 'icu_admit_source', 'icu_id', 'icu_stay_type',
       'icu_type', 'pre_icu_los_days', 'readmission_status', 'weight',
       'albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis',
       'apache_post_operative', 'arf_apache', 'bilirubin_apache', 'bun_apache',
       'creatinine_apache', 'fio2_apache', 'gcs_eyes_apache',
       'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache',
       'glucose_apache', 'heart_rate_apache', 'hematocrit_apache',
       'intubated_apache', 'map_apache', 'paco2_apache', 'paco2_for_ph_apache',
       'pao2_apache', 'ph_apache', 'resprate_apache', 'sodium_apache',
       'temp_apache', 'urineoutput_apache', 'ventilated_apache', 'wbc_apache',
       'aids', 'cirrhosis', 'hepatic_failure', 'immunosuppression', 'leukemia',
       'lymphoma', 'solid_tumor_with_metastasis', 'diabetes_mellit